In [1]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
from datetime import timedelta

# visualization
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# configure visualizations
sns.set_style('whitegrid')
figsize=(8,6)

In [2]:
def load_df(path, columns=None, use_threads=True):
    try:
        return pq.read_table(path, columns=columns, use_threads=use_threads).to_pandas()
    except Exception as e:
        print(e)

def downcast_ints(df):
    # downcast int types
    df_int = df.select_dtypes(include=['int'])
    converted_int = df_int.apply(pd.to_numeric, downcast='unsigned')
    df[converted_int.columns] = converted_int
    
    return df
        
def prepare(path,cols):

    df = load_df(path,cols)
    df = df.reset_index(drop=True)
    df = downcast_ints(df)
    
    return df


In [3]:
cols = ['sample_id', 'network_type', 'mobile_network_type', 'mobile_data_status', 'mobile_data_activity', 'roaming_enabled', 'wifi_status', 'wifi_signal_strength', 'wifi_link_speed', 'wifi_ap_status', 'network_operator', 'sim_operator', 'mcc', 'mnc']
df = prepare('1-parquet-files/network_details.parquet',cols)


#fix unsigned int
df_level = df.sample_id 
converted_level = df_level.astype(np.int32)
df['sample_id'] = converted_level
df_level = df.roaming_enabled
converted_level = df_level.astype(np.int32)
df['roaming_enabled'] = converted_level

df['network_type'] = df['network_type'].apply(lambda x: x.upper())
df['mobile_network_type'] = df['mobile_network_type'].apply(lambda x: x.upper())
#df['mobile_network_type'] = df['mobile_network_type'].apply(lambda x: 'UNKNOWN' if '0' else 'IWLEN' if '18' else 'GSM' if '16' else 'NR5G' if '20' else 'TD_SCDMA' if '17' else x)

df['mobile_data_status'] = df['mobile_data_status'].apply(lambda x: x.upper())
df['mobile_data_activity'] = df['mobile_data_activity'].apply(lambda x: x.upper())
df['wifi_status'] = df['wifi_status'].apply(lambda x: x.upper())
df['wifi_ap_status'] = df['wifi_ap_status'].apply(lambda x: x.upper())
df['network_operator'] = df['network_operator'].apply(lambda x: x.upper() if pd.notnull(x) else x)
df['sim_operator'] = df['sim_operator'].apply(lambda x: x.upper())

df['network_type'] = df['network_type'].apply(lambda x: 'BLUETOOTH TETHERING' if (x == 'BLUETOOTH_TETHER' ) else x)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23533845 entries, 0 to 23533844
Data columns (total 14 columns):
sample_id               int32
network_type            object
mobile_network_type     object
mobile_data_status      object
mobile_data_activity    object
roaming_enabled         int32
wifi_status             object
wifi_signal_strength    int64
wifi_link_speed         int64
wifi_ap_status          object
network_operator        object
sim_operator            object
mcc                     object
mnc                     object
dtypes: int32(2), int64(2), object(10)
memory usage: 2.3+ GB


In [5]:
df.to_parquet('2-datasets/network_details.parquet', compression='none') 

In [6]:
dfNetworkTypeGB = df.groupby(['network_type'])['network_type'].count().reset_index(name='count').sort_values(['count'], ascending=False).head(20)
print(dfNetworkTypeGB)

          network_type     count
5                 WIFI  11986480
4              UNKNOWN   6114227
2               MOBILE   5428354
0  BLUETOOTH TETHERING      3822
3              MOBILE2       952
1             ETHERNET        10
